In [8]:
# code to generate the C function for endmembers (pure species)
# Script for igneous database

# last update 23.04.23, NR

# Load needed libraries
# import nlopt
import numpy as np
import math as math
import cmath
import random

# Si 1
# Ti 2
# Al 3
# Cr 19
# Fe 4
# Mg 5
# Mn 6
# Ca 7
# Na 8
# K 9
# O 10
# H 11

directory = 'input_database_TC'
database  = 'tc-ds635.txt'
pathTC    = directory+'/'+database
out_name  = 'MAGEMin_'+database

bar2kbar  = 1e-3;

In [9]:
def func_print(data):
    txt ='{'
    for i,val in enumerate(data):
        if i == len(data)-1:
            txt += str(val)
        else:
            txt += str(val)+', ' 
    
    txt +='}'    
    return(txt)

In [10]:
# READ Perple_X db to add seismic data (+some new ones)

# ref0: 1 bar, 298.15 K
path      = './Perple_X/hp02ver.dat'
shift     = 7;
gemPP     = open(path, encoding = "ISO-8859-1")
lineP     = gemPP.readlines()

# create dictionary to store the shear modulus data along with the composition of the end-members
emPP      = {}
emTC      = {}

#read thermocalc endmember file and fill endmember structure
gemTC     = open(pathTC,'r')

for i in range(0,len(lineP)):

    if i > shift:
        words0 = lineP[i-shift].split();
        words1 = lineP[i+1-shift].split();
        words2 = lineP[i-1].split()
        words  = lineP[i].split()

        count0 = len(words0)
        count1 = len(words1)
        count2 = int(len(words2)/3)
        count  = len(words)
        
        if count >= 1:
            if words[0] == 'end' and count0 == 0 and count1 > 0:

                mu0     = [0.,0.,0.]
                n0      = lineP[i+1-shift].split()
                em_name = n0[0]
                
                c0      = lineP[i+2-shift].split()
                emPP[em_name] = c0;
                emPP[em_name].append(mu0)
                
                for j in range(0,count2):
                    emPP[em_name][1][j] = float(words2[(j+1)*3-1])
                    
mu0     = [0.,0.,0.]
emL2add = ['hemL','eskL','woL','ruL','H2O']
for i in emL2add:
    emPP[i] = ['-']
    emPP[i].append(mu0)    
    emPP[i][1][0] = 1e-16;
    
#add missing end-members 
emPP['kos']  = emPP['jd']   #Kosmochlor = jadeite. I could not find any new data on shear modulus.   

#ADD KNORRINGITE (Dymshits et al., 2014)
emPP['knor'] = ['Mg3Cr2Si3O12']
emPP['knor'].append([850000,1.4,-134])

#ADD Picrochromite (Zou et al 2013) -> used Mg2SiO4 !?
emPP['picr'] = ['MgCr2O4']
emPP['picr'].append([1090000,0.58,-140])

#ADD Quandelite (Zou et al 2013)  -> used MgFeSiO4 !?
emPP['qnd'] = ['(Mg,Fe3+)2(Ti,Fe3+,Al)O4']
emPP['qnd'].append([1190000,1.2,-150])

# READ TC database
new_content = [[] for x in range(4)] #create 2d array with 4 lines and unspecified column number

c=1;
for lineT in gemTC:
    words = lineT.split()
    count = len(words)

    if c == 1: #attribute endmember name to the dictionary
        name0 = words[0]
        content = np.asarray(words[1:count])   
        new_content[0] = [float(i) for i in content]
    elif c > 1 and c < 5: #attribute thermodynamic data to the endmember dictionary entry
        content = np.asarray(words[0:count]) 
        new_content[c-1] = [float(i) for i in content]  

    emTC[name0] = new_content[0:4]

    c+=1
    if c == 5:
        c = 1
        
        
for i in emPP:
    if (emPP[i][1][0] > 1e-15):
        emPP[i][1][0] *= bar2kbar;
        emPP[i][1][1] *= bar2kbar;

In [11]:
#first clean up the end-member file, remove first and last lines of the file

gem = open(pathTC,'r')

#declare endmember structure, name + thermodynamic data
em = {}
new_content = [[] for x in range(4)] #create 2d array with 4 lines and unspecified column number

#read thermocalc endmember file and fill endmember structure
c=1;
for line in gem:
    words = line.split()
    count = len(words)

    if c == 1: #attribute endmember name to the dictionary
        name0 = words[0]
        content = np.asarray(words[1:count])   
        new_content[0] = [float(i) for i in content]
    elif c > 1 and c < 5: #attribute thermodynamic data to the endmember dictionary entry
        content = np.asarray(words[0:count]) 
        new_content[c-1] = [float(i) for i in content]  

    em[name0] = new_content[0:4]

    c+=1
    if c == 5:
        c = 1
#"SiO2","Al2O3","CaO","MgO","FeO","K2O","Na2O","TiO2","O","Cr2O3","H2O"      
apo = [3.0,5.0,2.0,2.0,2.0,3.0,3.0,3.0,1.0,5.0,3.0]; #number of atoms per oxide

# generate the C function
sp = '    '
op = ''
ip = ''
ip += 'char em_list['+str(len(em)-1)+'][5] = {'
for ix,i in enumerate(em):
    if ix <(len(em)-1):
        ip += '"' + str(i)+'", '
    else:
        ip += '"' + str(i)+'"'
ip += '};'

op += 'struct EM_db arr_em_db['+str(len(em))+'] = {\n'

for ix,i in enumerate(em):
    op += sp +  '{\n'
  
    op += sp + sp + '"' + str(i)+'",\n'
    for jx,j in enumerate(em[i]):
        if jx == 0:
            #Chemical composition
            el_comp = np.zeros(19);
            chem_comp = np.zeros(11);

            y  = 1;
            l = len(em[i][0])-1;

            while y < l:
                el_comp[int(em[i][0][y])-1] = em[i][0][y+1];
                y = y + 2;

            #Normalization of endmember composition
            chem_comp = [el_comp[0], el_comp[2]/2, el_comp[6], el_comp[4], el_comp[3], el_comp[8]/2,el_comp[7]/2, el_comp[1],el_comp[9], el_comp[18]/2,el_comp[10]/2];

            if 2*chem_comp[0] + 3*chem_comp[1] + chem_comp[2] + chem_comp[3] + chem_comp[4] + chem_comp[5] +  chem_comp[6] + 2*chem_comp[7] + 3*chem_comp[9]+ chem_comp[10] != el_comp[8]:
                chem_comp[8] =  chem_comp[8] - (2*chem_comp[0] + 3*chem_comp[1] + chem_comp[2] + chem_comp[3] + chem_comp[4] + chem_comp[5] +  chem_comp[6] + 2*chem_comp[7] + 3*chem_comp[9]+ chem_comp[10]);
            
            apf = sum(np.array(chem_comp)*np.array(apo))
            chem_comp.append(apf)
            
            op += sp+ sp +func_print(chem_comp)+',\n'
            
        elif jx < 3 and jx > 0:
            if jx == 1:
                op += sp+ sp +func_print(j)+',\n'
            if jx == 2:
                op += sp+ sp +func_print(j)+',\n' 
        else:
            third_line = [0.0]*11
            for k,val in enumerate(j):
                third_line[k] = val
            op += sp+ sp +func_print(third_line)+',\n'
   

    if i in emPP:
        op += sp + sp + '{'
        for j in range(0,3):  
            if j < 2:
                op += str(emPP[i][1][j])+','
            else:
                op += str(emPP[i][1][j])
        op += '}\n'
    else:
        op += sp + sp + '{'
        for j in range(0,3):  
            if j < 2:
                op += '0.0,'
            else:
                op += '0.0'
        op += '}\n'
    
    
            
    if ix < len(em)-1:
        op += sp + '},\n'
    else:
        op += sp + '}\n'
        
op += '};\n\n'


In [12]:
print(op)
text_file   = open(out_name, "w")
n           = text_file.write(op)

struct EM_db arr_em_db[291] = {
    {
        "fo",
        {1.0, 0.0, 0.0, 2.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 7.0},
        {-2172.452, 0.0951, 4.366},
        {0.2333, 1.494e-06, -603.8, -1.8697},
        {2.85e-05, 1285.0, 3.84, -0.00299, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0},
        {810.0,0.00182,-140.0}
    },
    {
        "fa",
        {1.0, 0.0, 0.0, 0.0, 2.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 7.0},
        {-1477.337, 0.151, 4.631},
        {0.2011, 1.733e-05, -1960.6, -0.9009},
        {2.82e-05, 1256.0, 4.68, -0.00373, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0},
        {510.0,0.00062,-108.0}
    },
    {
        "teph",
        {1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 2.0, 0.0, 0.0, 5.0},
        {-1733.842, 0.1559, 4.899},
        {0.2196, 0.0, -1292.7, -1.3083},
        {2.86e-05, 1256.0, 4.68, -0.00373, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0},
        {510.0,0.00062,-108.0}
    },
    {
        "lrn",
        {1.0, 0.0, 2.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 7.0},
        {-230